In [1]:
import os
import sys

print("Current Dir:", os.getcwd())
sys.path.append(os.getcwd())
print("sys.path =", sys.path)


Current Dir: /mnt/c/home/chieetan/diffusion-model-for-arm-public
sys.path = ['/home/chieetan/.pyenv/versions/3.10.13/lib/python310.zip', '/home/chieetan/.pyenv/versions/3.10.13/lib/python3.10', '/home/chieetan/.pyenv/versions/3.10.13/lib/python3.10/lib-dynload', '', '/mnt/c/home/chieetan/diffusion-model-for-arm-public/venv/lib/python3.10/site-packages', '/mnt/c/home/chieetan/diffusion-model-for-arm-public']


In [2]:
from simulator import definition as armdef
print(armdef)  # 成功すれば <module 'simulator.definition' from ...> が出る


<module 'simulator.definition' from '/mnt/c/home/chieetan/diffusion-model-for-arm-public/simulator/definition.py'>


In [3]:
from diffusion_model.model import Dataset


In [4]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
!pip install pygame
import sys
import os

# diffusion_model-for-arm-public を sys.path に追加
# sys.path.append(os.path.abspath("."))  # 現在位置が diffusion-model-for-arm-public ならこれでOK
# または
sys.path.append(os.path.abspath("./diffusion_model"))  # model.pyから見て正しいならこちら

# モジュールが読み込めるか確認
from simulator import definition as armdef



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import sys
!{sys.executable} -m pip install pygame

from diffusion_model import Dataset, EarlyStopping, Model, gen_xt, normalize, steps


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
# Notebookの先頭でこれを実行
import sys
import os
sys.path.append(os.path.abspath("."))  # diffusion-model-for-arm-public を含む

# 確認：これでmodel.pyをimportできるようになるはず
from diffusion_model import Dataset, EarlyStopping, Model, gen_xt, normalize, steps


In [7]:
import sys
print(sys.path)


['/home/chieetan/.pyenv/versions/3.10.13/lib/python310.zip', '/home/chieetan/.pyenv/versions/3.10.13/lib/python3.10', '/home/chieetan/.pyenv/versions/3.10.13/lib/python3.10/lib-dynload', '', '/mnt/c/home/chieetan/diffusion-model-for-arm-public/venv/lib/python3.10/site-packages', '/mnt/c/home/chieetan/diffusion-model-for-arm-public', '/mnt/c/home/chieetan/diffusion-model-for-arm-public/diffusion_model', '/mnt/c/home/chieetan/diffusion-model-for-arm-public']


In [8]:
# 例（venv使用時）
!python3 -mpip install ipykernel
!python3 -m ipykernel install --user --name myenv --display-name "Python (myenv)"



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Installed kernelspec myenv in /home/chieetan/.local/share/jupyter/kernels/myenv


In [9]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # None ならCPU版


2.6.0+cu124
12.4


In [10]:
# PyTorch CUDA対応版をインストール（例: CUDA 11.8対応のPyTorch）
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.0%2Bcu118-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.0%2Bcu118-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_cupti_cu11-11.8.87-py3-none-manylinux1_x86_64.whl (13.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cudnn

In [11]:
#データセットの読み込み
device = "cuda"
dataset = Dataset("data/train.csv")
model = Model(steps).to(device)

In [12]:
batch_size = 100
# データローダーの初期化
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=14,
        drop_last=True,
        pin_memory=True,
    )
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(
        dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )

batch_size = 64
train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
    )
val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

/mnt/c/home/chieetan/diffusion-model-for-arm-public/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 14 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
epochs = 50
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        start_factor=1,
        end_factor=0.1,
        total_iters=epochs,
    )
criterion = nn.MSELoss()
# EarlyStoppingクラスを使用して、学習の早期終了を設定
early_stopping = EarlyStopping(
        # patienceを5に設定
        patience=5,
        # min_deltaを0.001に設定
        delta=0.01,
    )

In [ ]:
# 学習を開始
for epoch in tqdm(range(epochs)):
    model.train()
    total_loss = 0
    # 各バッチを(x, pos, theta)という形式のタプルとして取得し、enumerateメソッドを使用してデータローダーからバッチを順番に取得し、各バッチに対してインデックスを付ける。
    for batch, (x, pos, theta) in tqdm(enumerate(dataloader)):
        # データをGPUに転送
        x, pos, theta = x.to(device), pos.to(device), theta.to(device)
        # xを正規化
        x = normalize(x)
        print(f"x.shape = {x.shape}")
        # torch.randintメソッドを使用して、1からsteps(25)までの値をランダムに生成し、バッチサイズ（100）行1列のテンソルを生成
        t = torch.randint(1, steps, (x.shape[0],), device=device).long()
        # torch.randn_likeメソッドを使用して、正規分布に従うランダムな値を持つxと同じサイズのテンソルを生成
        y = torch.randn_like(x).to(device)
        # gen_xtメソッド(ノイズを生成）を使用して、x, t, yを引数として渡し、xを生成
        x = gen_xt(x, t, y)
        # x及びyをデバイスに転送
        x, y = x.to(device), y.to(device)
        # モデルにx, t, posを引数として渡し、予測値を取得
        pred = model(x, t, pos)
        # 平均二乗誤差損失関数を使用して、予測値とyの損失を計算
        loss = criterion(pred, y)
        # 勾配を初期化
        optimizer.zero_grad()
        # 逆伝播を実行
        loss.backward()
        # 学習率と最適化手法に基づいて、パラメータを更新
        optimizer.step()
        # epoch全体の損失を、各バッチの損失を累積することによって計算
        total_loss += loss.item()
    print(f"Epoch {epoch}: train_loss = {total_loss / len(dataloader):.4f}")
    scheduler.step()
    # 検証
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val, pos_val, theta_val in val_dataloader:
            x_val, pos_val, theta_val = (
                    x_val.to(device),
                    pos_val.to(device),
                    theta_val.to(device),
                )
            x_val = normalize(x_val)
            t_val = torch.randint(1, steps, (x_val.size(0),), device=device).long()
            y_val = torch.randn_like(x_val).to(device)
            x_val = gen_xt(x_val, t_val, y_val)

            pred_val = model(x_val, t_val, pos_val)
            loss_val = criterion(pred_val, y_val)
            val_loss += loss_val.item()
        val_loss /= len(val_dataloader)

  0% 0/50 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0% 0/50 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (64) must match the size of tensor b (100) at non-singleton dimension 0